## A. IMPORT STATEMENTS

In [1]:
# Import Statements
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import wandb
import random

## B. SET UP FOR DEVICE AGNOSTIC CODE

In [2]:
# Check if GPU is available
torch.cuda.is_available()

True

In [3]:
# Setup device agnostic code
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

## C. DATA LOADING AND PREPROCESSING STEPS
### NOTE:- I have unzipped the inaturalist_12K dataset and uploaded to the Google Drive for easier access.

In [4]:
# Mount the google drive (RUN ONLY IN COLAB)
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
# Set up data directory path
TRAIN_DATA_DIR = "/content/drive/My Drive/Data/inaturalist_12K/train"
TEST_DATA_DIR = "/content/drive/My Drive/Data/inaturalist_12K/val"

In [ ]:
def data_transformations(image_size=(224, 224), data_augment=False):

  """This function returns data transformations for the images data."""

  # Define transformations to be applied (Base Transformations)
  transformations = [
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]) # NOTE:- I am planning to use EfficientNetV2 for part B of the assignment so using the same values as used for that network
  ]

  # If Augmentation is needed, add them to transform list
  if data_augment:
    transformations += [
      transforms.RandomHorizontalFlip(0.05),
      transforms.RandomVerticalFlip(0.05),
      transforms.RandomRotation(degrees=20),
      transforms.RandomApply([transforms.ColorJitter(0.2, 0.2, 0.2, 0.2)], p=0.05),
      transforms.RandomApply([transforms.GaussianBlur(3)], p=0.05)
    ]

  transformer = transforms.Compose(transformations)

  return transformer


def get_train_and_val_dataloaders(train_data_dir, image_size=(224, 224), data_augment=False, valset_size=0.2, batch_size=16):

  """This function returns the dataloader for trainset and validation set and classnames"""

  transformer = data_transformations(image_size, data_augment)

  # Dowload the total_train dataset
  total_trainset = torchvision.datasets.ImageFolder(root = train_data_dir, transform=transformer)

  # Get the classnames
  classnames = total_trainset.classes

  # Split the total_train data into train data and val data
  labels = [label for _, label in total_trainset.samples]

  if valset_size != 0:
    train_indices, val_indices = train_test_split(
                                    range(len(total_trainset)),
                                    test_size=valset_size,
                                    stratify=labels,
                                    random_state=42
                                    )
  else:
    train_indices = range(len(total_trainset))
    val_indices = []

  # Create the trainset and valset
  trainset = torch.utils.data.Subset(total_trainset, train_indices)
  valset = torch.utils.data.Subset(total_trainset, val_indices)

  # Create the dataloaders
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

  return trainloader, valloader, classnames



def get_test_dataloader(test_data_dir, image_size=(224, 224), batch_size=16):

  """This function returns the test dataloader"""

  transformer = data_transformations(image_size, False)

  #Download the test data
  testset = torchvision.datasets.ImageFolder(root=test_data_dir, transform=transformer)



  testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

  return testloader

## D. MODEL CLASS
The model defined below is a CNN Model class with the help of PyTorch nn.Module.
Model Specifications:-
1. 5 Convolution Blocks
2. 1 Fully connected layer
3. 1 output layer
4. Number of filters, kernel sizes, fully connected layer size, etc. are modular.

In [8]:
class CNNModel(nn.Module):
  def __init__(self, image_size, in_channels=3, num_classes=10,
                num_filters=[64, 64, 64, 64, 64], kernel_size=[3, 3, 3, 3, 3],
                activation_fn=nn.ReLU, fc_layer_size=2048,
                batchnorm=False, dropout=0.0):

    super().__init__()

    h, w = image_size

    # Block 1
    self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=num_filters[0], kernel_size=kernel_size[0])
    h, w = h - kernel_size[0] + 1, w - kernel_size[0] + 1
    self.batchnorm1 = nn.BatchNorm2d(num_filters[0]) if batchnorm else nn.Identity()
    self.activation1 = activation_fn()
    self.dropout1 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
    self.maxpool1 = nn.MaxPool2d(2, 2)
    h, w = h//2, w//2

    # Block 2
    self.conv2 = nn.Conv2d(in_channels=num_filters[0], out_channels=num_filters[1], kernel_size=kernel_size[1])
    h, w = h - kernel_size[1] + 1, w - kernel_size[1] + 1
    self.batchnorm2 = nn.BatchNorm2d(num_filters[1]) if batchnorm else nn.Identity()
    self.activation2 = activation_fn()
    self.dropout2 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
    self.maxpool2 = nn.MaxPool2d(2, 2)
    h, w = h//2, w//2

    # Block 3
    self.conv3 = nn.Conv2d(in_channels=num_filters[1], out_channels=num_filters[2], kernel_size=kernel_size[2])
    h, w = h - kernel_size[2] + 1, w - kernel_size[2] + 1
    self.batchnorm3 = nn.BatchNorm2d(num_filters[2]) if batchnorm else nn.Identity()
    self.activation3 = activation_fn()
    self.dropout3 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
    self.maxpool3 = nn.MaxPool2d(2, 2)
    h, w = h//2, w//2

    # Block 4
    self.conv4 = nn.Conv2d(in_channels=num_filters[2], out_channels=num_filters[3], kernel_size=kernel_size[3])
    h, w = h - kernel_size[3] + 1, w - kernel_size[3] + 1
    self.batchnorm4 = nn.BatchNorm2d(num_filters[3]) if batchnorm else nn.Identity()
    self.activation4 = activation_fn()
    self.dropout4 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
    self.maxpool4 = nn.MaxPool2d(2, 2)
    h, w = h//2, w//2

    # Block 5
    self.conv5 = nn.Conv2d(in_channels=num_filters[3], out_channels=num_filters[4], kernel_size=kernel_size[4])
    h, w = h - kernel_size[4] + 1, w - kernel_size[4] + 1
    self.batchnorm5 = nn.BatchNorm2d(num_filters[4]) if batchnorm else nn.Identity()
    self.activation5 = activation_fn()
    self.dropout5 = nn.Dropout2d(dropout) if dropout!=0 else nn.Identity()
    self.maxpool5 = nn.MaxPool2d(2, 2)
    h, w = h//2, w//2

    # Flattening layer
    self.flatten = nn.Flatten()

    # Fully connected layer
    self.fc_layer = nn.Linear(in_features=num_filters[4] * h * w, out_features=fc_layer_size)
    self.batchnorm_fc = nn.BatchNorm1d(fc_layer_size) if batchnorm else nn.Identity()
    self.act_fc = activation_fn()
    self.drop_fc = nn.Dropout(dropout) if dropout!=0 else nn.Identity()

    # Output layer
    self.out = nn.Linear(in_features=fc_layer_size, out_features=num_classes)

  def forward(self, x):
    # Block 1
    x = self.conv1(x)
    x = self.batchnorm1(x)
    x = self.activation1(x)
    x = self.dropout1(x)
    x = self.maxpool1(x)

    # Block 2
    x = self.conv2(x)
    x = self.batchnorm2(x)
    x = self.activation2(x)
    x = self.dropout2(x)
    x = self.maxpool2(x)

    # Block 3
    x = self.conv3(x)
    x = self.batchnorm3(x)
    x = self.activation3(x)
    x = self.dropout3(x)
    x = self.maxpool3(x)

    # Block 4
    x = self.conv4(x)
    x = self.batchnorm4(x)
    x = self.activation4(x)
    x = self.dropout4(x)
    x = self.maxpool4(x)

    # Block 5
    x = self.conv5(x)
    x = self.batchnorm5(x)
    x = self.activation5(x)
    x = self.dropout5(x)
    x = self.maxpool5(x)

    # Flatten
    x = self.flatten(x)

    # Fully connected layers
    x = self.fc_layer(x)
    x = self.batchnorm_fc(x)
    x = self.act_fc(x)
    x = self.drop_fc(x)

    # Output layer
    x = self.out(x)

    return x

## E. MODEL TRAINING
The "train_model" method defined below trains the model on the trainloader and then evaluate on valloader for the specified number of epochs.

In [9]:
def train_model(model, trainloader, valloader, criterion, optimizer, device, epochs=10):

  """This function trains and evaluates the model for the specified number of epochs."""

  # Metrics to keep track of
  train_epoch_losses = []
  train_epoch_accuracies = []
  val_epoch_losses = []
  val_epoch_accuracies = []

  for epoch in range(epochs):

    # Set the model in train mode
    model.train()

    # Epoch Metrics
    train_running_loss = 0
    train_correct = 0
    train_total = 0

    for data in tqdm(trainloader):

      inputs, labels = data[0].to(device), data[1].to(device)

      optimizer.zero_grad()

      outputs = model(inputs)

      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      # Update metrics
      train_running_loss += loss.item() * inputs.size(0)
      train_total += inputs.size(0)
      train_correct += torch.sum(labels == torch.argmax(outputs, dim=1)).item()

    train_epoch_loss = train_running_loss / train_total
    train_epoch_accuracy = 100 * train_correct / train_total

    train_epoch_losses.append(train_epoch_loss)
    train_epoch_accuracies.append(train_epoch_accuracy)

    # Set the model to evaluation mode
    model.eval()

    # Epoch Metrics
    val_running_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
      for data in tqdm(valloader):

        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Update Metrics
        val_running_loss += loss.item() * inputs.size(0)
        val_total += inputs.size(0)
        val_correct += torch.sum(labels == torch.argmax(outputs, dim=1)).item()

    val_epoch_loss = val_running_loss / val_total
    val_epoch_accuracy = 100 * val_correct / val_total

    val_epoch_losses.append(val_epoch_loss)
    val_epoch_accuracies.append(val_epoch_accuracy)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_epoch_loss:.3f}, Train Acc: {train_epoch_accuracy:.2f}%, Val Loss: {val_epoch_loss:.3f}, Val Acc: {val_epoch_accuracy:.2f}%")

  return train_epoch_losses, train_epoch_accuracies, val_epoch_losses, val_epoch_accuracies

## F. TRIAL RUN

In [7]:
# Image dimension
image_size = (224, 224)

In [ ]:
# Get the trainloader, valloader and classnames
trainloader, valloader, classnames = get_train_and_val_dataloaders(TRAIN_DATA_DIR, image_size=image_size)

In [ ]:
# Define a basic model and move to the device being used
model = CNNModel(image_size)
model.to(device)

CNNModel(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm1): Identity()
  (activation1): ReLU()
  (dropout1): Identity()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm2): Identity()
  (activation2): ReLU()
  (dropout2): Identity()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm3): Identity()
  (activation3): ReLU()
  (dropout3): Identity()
  (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm4): Identity()
  (activation4): ReLU()
  (dropout4): Identity()
  (maxpool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (batchnorm5): Identity()

In [ ]:
# Define the criterion and optimizer to be used in training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [ ]:
# Train the model
train_model(model, trainloader, valloader, criterion, optimizer, device, 10)

100%|██████████| 126/126 [05:55<00:00,  2.82s/it]


Epoch 1/10, Train Loss: 2.304, Train Acc: 9.49%, Val Loss: 2.303, Val Acc: 10.09%


100%|██████████| 126/126 [00:25<00:00,  4.93it/s]


Epoch 2/10, Train Loss: 2.304, Train Acc: 9.49%, Val Loss: 2.303, Val Acc: 9.99%


100%|██████████| 126/126 [00:23<00:00,  5.27it/s]


...

100%|██████████| 126/126 [00:25<00:00,  5.02it/s]


Epoch 9/10, Train Loss: 2.304, Train Acc: 9.27%, Val Loss: 2.303, Val Acc: 9.99%


100%|██████████| 126/126 [00:26<00:00,  4.79it/s]

Epoch 10/10, Train Loss: 2.304, Train Acc: 9.87%, Val Loss: 2.303, Val Acc: 9.99%


([2.3038672651018826,
  2.3038257285710606,
  2.3040455661652657,
  2.3039399892605457,
  2.303906606252033,
  2.3039464572355683,
  2.30354961865311,
  2.3036965939690703,
  2.3040666415219064,
  2.3037933534960016],
 [9.492880339745192,
  9.492880339745192,
  9.330502123407445,
  9.417936547589308,
  9.380464651511366,
  9.305520859355484,
  10.042468148888334,
  9.767674244316762,
  9.268048963277542,
  9.867599300524606],
 [2.303045203754833,
  2.302984376530071,
  2.302938708058604,
  2.3029538615719303,
  2.302663381759461,
  2.303031295924992,
  2.303070899847147,
  2.3033623700137142,
  2.302898993144383,
  2.3032914591835927],
 [10.08991008991009,
  9.99000999000999,
  9.99000999000999,
  9.99000999000999,
  9.99000999000999,
  9.99000999000999,
  10.08991008991009,
  9.99000999000999,
  9.99000999000999,
  9.99000999000999])

### **OBSERVATIONS:-** For the trial run the model didn't perform well, giving the accuracy of ~10% for a train class classification, which is as good as random guessing.

## G. SOME HELPER FUNCTIONS

In [1]:
def get_activation_function(name):

  """This function returns the activation function based on the name passed"""

  if name == "leaky_relu":
    return nn.LeakyReLU
  elif name == "parametric_relu":
    return nn.PReLU
  elif name == "gelu":
    return nn.GELU
  elif name == "silu":
    return nn.SiLU
  elif name == "mish":
    return nn.Mish
  return nn.ReLU



def get_num_filters(name):

  """This function is a helper function for the hyper parameter sweep.
     As we need different filter sizes for the model to perform the sweep."""

  if name == "equal16":
    return [16, 16, 16, 16, 16]
  elif name == "equal32":
    return [32, 32, 32, 32, 32]
  elif name == "equal64":
    return [64, 64, 64, 64, 64]
  elif name == "doubling16":
    return [16, 32, 64, 128, 256]
  elif name == "doubling32":
    return [32, 64, 128, 256, 512]
  elif name == "halving256":
    return [256, 128, 64, 32, 16]
  else:
    return [100, 80, 50, 80, 100]



def get_kernel_size(name):

  """This function is a helper function for the hyper parameter sweep.
     As we need different kernel sizes for the model to perform the sweep."""

  if name == "constant5":
    return [5, 5, 5, 5, 5]
  elif name == "constant7":
    return [7, 7, 7, 7, 7]
  elif name == "decreasing":
    return [5, 5, 3, 3, 1]
  elif name == "increasing":
    return [1, 3, 3, 5, 5]
  return [3, 3, 3, 3, 3]



## H. WANDB HYPERPARAMETER SWEEP

In [ ]:
def sweep_hyperparameters(config=None):

  with wandb.init(config=config):

    config = wandb.config
    wandb.run.name = f"activation_{str(config.activation)}_filters_{str(config.num_filters)}_lr_{config.learning_rate}_kernel_{config.kernel_size}_fc_size_{config.fc_layer_size}"

    # Log in my details
    wandb.config.update({"NAME": "SIDDHANT BARANWAL", "ROLL NO.": "DA24M021"})

    image_size=(224, 224)

    TRAIN_DATA_DIR = "/content/drive/My Drive/Data/inaturalist_12K/train"
    trainloader, valloader, classnames = get_train_and_val_dataloaders(TRAIN_DATA_DIR, image_size=image_size, data_augment=config.data_augment, valset_size=0.2, batch_size=config.batch_size)


    model = CNNModel(image_size, num_filters=get_num_filters(config.num_filters), kernel_size=get_kernel_size(config.kernel_size),
                     activation_fn=get_activation_function(config.activation), batchnorm=config.batch_norm, dropout=config.dropout,
                     fc_layer_size=config.fc_layer_size)
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    train_losses, train_accuracies, val_losses, val_accuracies = train_model(model, trainloader, valloader, criterion, optimizer, device, epochs=10)

    # Log the evaluation metrics

    for epoch in range(len(train_losses)):
      wandb.log({
          "epoch": epoch,
          "train_loss": train_losses[epoch],
          "train_accuracy": train_accuracies[epoch],
          "validation_loss": val_losses[epoch],
          "validation_accuracy": val_accuracies[epoch]
      })

    wandb.log({
        "val_accuracy": val_accuracies[-1]
    })

In [ ]:
sweep_config = {
    "method" : "bayes",
    "metric" : {"name": "val_accuracy", "goal": "maximize"},
    "parameters" : {
        "data_augment" : {"values" : [True, False]},
        "batch_norm" : {"values" : [True, False]},
        "dropout" : {"values" : [0.0, 0.2, 0.4]},
        "learning_rate" : {"values" : [0.01, 0.001, 0.0005, 0.0001]},
        "activation" : {"values" : ["relu", "leaky_relu", "parametric_relu",
                                    "gelu", "silu", "mish"]},
        "num_filters" : {"values" : ["equal16", "equal32", "equal64", "doubling16", "doubling32", "halving256"]},
        "kernel_size" : {"values" : ["constant3", "constant5", "constant7", "decreasing", "increasing"]},
        "fc_layer_size" : {"values": [2048, 1024, 512]},
        "batch_size": {"values": [8, 16, 32]}
    }
}

sweep_id = wandb.sweep(sweep_config, project = "da24m021_da6401_assignment2")
wandb.agent(sweep_id, function = sweep_hyperparameters, count = 50)

Create sweep with ID: k2txx1xm
Sweep URL: https://wandb.ai/da24m021-indian-institute-of-technology-madras/da24m021_da6401_assignment2/sweeps/k2txx1xm


wandb: Agent Starting Run: cerulhgd with config:
wandb: 	activation: leaky_relu
wandb: 	batch_norm: False
wandb: 	batch_size: 8
wandb: 	data_augment: False
wandb: 	dropout: 0.2
wandb: 	fc_layer_size: 512
wandb: 	kernel_size: constant5
wandb: 	learning_rate: 0.01
wandb: 	num_filters: halving256


100%|██████████| 251/251 [00:27<00:00,  8.99it/s]


Epoch 1/10, Train Loss: 107038537.954, Train Acc: 10.30%, Val Loss: 2349336.981, Val Acc: 10.69%


100%|██████████| 251/251 [00:28<00:00,  8.85it/s]


...

100%|██████████| 126/126 [00:32<00:00,  3.94it/s]

Epoch 10/10, Train Loss: 1.901, Train Acc: 32.41%, Val Loss: 1.975, Val Acc: 30.47%


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▅▆▆▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁
validation_accuracy,▁▃▅▆▆▇██▇█
validation_loss,█▆▅▃▃▂▂▁▂▂
epoch,9
train_accuracy,32.41319
train_loss,1.90146
val_accuracy,30.46953
validation_accuracy,30.46953


wandb: Agent Starting Run: kfnecoz5 with config:
wandb: 	activation: gelu
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augment: False
wandb: 	dropout: 0
wandb: 	fc_layer_size: 2048
wandb: 	kernel_size: increasing
wandb: 	learning_rate: 0.0001
wandb: 	num_filters: doubling16


 78%|███████▊  | 49/63 [00:20<00:05,  2.55it/s]

## **NOTE :-** The hyperparameter sweep was set for 50 experiments. But due to colab rate limits, the sweep got disconnected after 16 runs. But we got to see some satisfactory results within these 16 runs.




## The best validation accuracy I was able to get by hyperparameter tuning is 36.713.

Best hyperparameters among these are:-

| Rank | Activation | Batch Norm | Batch Size | Data Augment | Dropout | FC Layer Size | Kernel Size  | Learning Rate | Num Filters | Train Acc | Val Acc | Train Loss | Val Loss |
|------|------------|------------|------------|--------------|---------|---------------|--------------|---------------|-------------|-----------|---------|------------|---------|
| 1st  | Mish       | True       | 16         | True         | 0       | 2048          | Decreasing   | 0.0001        | Equal32     | 43.99%    | 36.71%  | 1.59254    | 1.84438 |
| 2nd  | Mish       | False      | 32         | False        | 0       | 2048          | Decreasing   | 0.0001        | Equal64     | 47.16%    | 36.11%  | 1.50012    | 1.84692 |
| 3rd  | Mish       | False      | 32         | False        | 0       | 2048          | Increasing   | 0.0001        | Equal64     | 36.62%    | 34.17%  | 1.80000    | 1.88000 |


## Observation:-
In one of the run I was also able to attain 95.89% accuracy on the train dataset with only 29.47% accuracy on validation set signifying considerable overfitting.

## I. BEST MODEL TRAINING AND EVALUATION

In [18]:
# Image resize size
image_size = (224, 224)


# Get the trainloader with complete train dataset (No validation set)
trainloader, _, classnames = get_train_and_val_dataloaders(TRAIN_DATA_DIR, image_size=image_size, valset_size=0, data_augment=True, batch_size=16)


# Define the model and move to the device
best_model = CNNModel(image_size, num_filters=get_num_filters("equal32"), kernel_size=get_kernel_size("decreasing"),
                     activation_fn=get_activation_function("mish"), batchnorm=True, dropout=0, fc_layer_size=2048)
best_model.to(device)

# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(best_model.parameters(), lr=0.0001)


epochs = 10

# Training Loops
for epoch in range(epochs):

  # Set the model in train mode
  best_model.train()

  # Metrics to keep track of
  running_loss = 0
  correct = 0
  total = 0

  for data in tqdm(trainloader):

    inputs, labels = data[0].to(device), data[1].to(device)

    optimizer.zero_grad()

    outputs = best_model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()
    optimizer.step()

    # Metric update
    running_loss += loss.item() * inputs.size(0)
    total += inputs.size(0)
    correct += torch.sum(labels == torch.argmax(outputs, dim=1)).item()

  train_epoch_loss = running_loss / total
  train_epoch_accuracy = 100 * correct / total

  print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_epoch_loss:.3f}, Train Acc: {train_epoch_accuracy:.2f}%")

100%|██████████| 626/626 [33:50<00:00,  3.24s/it]


Epoch 1/10, Train Loss: 2.102, Train Acc: 25.41%


100%|██████████| 626/626 [02:41<00:00,  3.87it/s]


Epoch 2/10, Train Loss: 1.953, Train Acc: 31.23%


100%|██████████| 626/626 [02:39<00:00,  3.92it/s]


...

Epoch 8/10, Train Loss: 1.665, Train Acc: 42.26%


100%|██████████| 626/626 [02:37<00:00,  3.98it/s]


Epoch 9/10, Train Loss: 1.647, Train Acc: 42.84%


100%|██████████| 626/626 [02:36<00:00,  4.00it/s]

Epoch 10/10, Train Loss: 1.608, Train Acc: 44.47%


In [28]:
# Best model evaluation

# Load the test dataset
testloader = get_test_dataloader(TEST_DATA_DIR, image_size=image_size, batch_size=16)

# Set the model in evaluation mode
best_model.eval()

# Values to keep track of
running_loss = 0
correct = 0
total = 0

with torch.no_grad():
  for data in tqdm(testloader):

    inputs, labels = data[0].to(device), data[1].to(device)
    outputs = best_model(inputs)
    loss = criterion(outputs, labels)

    running_loss += loss.item() * inputs.size(0)
    total += inputs.size(0)
    correct += torch.sum(labels == torch.argmax(outputs, dim=1)).item()

test_loss = running_loss / total
test_accuracy = (correct / total) * 100

print(f"\nTest Loss: {test_loss:.3f}, Test Acc: {test_accuracy:.2f}%")

100%|██████████| 125/125 [00:24<00:00,  5.10it/s]


Test Loss: 1.756, Test Acc: 40.50%


## Metrics of the best model :-
## Test Loss of best model :- 1.756
## Test Accuracy of best model :- 40.50%

## J. SAVE THE MODEL FOR FURTHER USE

In [24]:
model_path = "/content/drive/MyDrive/Data/models/da6401_2_best_model.pth"
torch.save(best_model, model_path)

In [26]:
best_model = torch.load(model_path, weights_only=False)

## K. Create the Prediction Table

In [33]:
# Get 10 random samples from test dataset
num_samples = 10
indices = random.sample(range(len(testloader.dataset)), num_samples)
sample_images = [testloader.dataset[i] for i in indices]

images = []
labels = []
predictions = []

for data in sample_images:
  input, label = data[0].to(device), data[1]
  input = input.unsqueeze(0)

  output = best_model(input)
  prediction = output.argmax(dim=1).item()

  # Undo image transformations
  image = input.squeeze(0).cpu().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
  image = np.clip(image, 0, 1)

  images.append(image)
  labels.append(classnames[label])
  predictions.append(classnames[prediction])

wandb.init(project = "da24m021_da6401_assignment2", name="prediction_visualization")
columns = ["Image", "True Label", "Predicted Label"]
prediction_table = wandb.Table(columns=columns)

for i in range(num_samples):
  img = wandb.Image(images[i])

  true_label = labels[i]
  pred_label = predictions[i]
  prediction_table.add_data(
      img,
      true_label,
      pred_label
  )

# Log the table to wandb
wandb.log({"Prediction Samples": prediction_table})